In [23]:
import torch
import torch.nn as nn

input = torch.ones((2,1,7,3))
conv2d = nn.Conv2d(1,8,(2,3))
res = conv2d(input)
print(res.shape)

"""
参数解释
--input的参数：(2,1,7,3) -> (batch_size, channels, height_1, width_1)
    batch_size=2：一个batch中的样例数
    channels=1：通道数，也就是当前层的深度
    height_1=7：也就是当前图片的高
    width_1=3：也就是当前图片的宽
    
--Conv2d的参数：(1,8,(2,3))  -> (in_channels, out_channels, (height_2, width_2))
    in_channels=1：输入通道数，和上面的input的channels一样
    out_channels=8：输出的深度，也就是卷积产生的通道数
    height_2=2：过滤器或者卷积核的高
    width_2=2：过滤器或者卷积核的宽

--res输出的参数：(2,8,6,1) -> (batch_size, out_channels, height_3, width_3)
    batch_size=2：一个batch中的样例数
    out_channels=8：输出通道数，也就是卷积核产生的通道数
    height_3=6：卷积结果的高度 = height_1 - height_2 + 1 = 7-2+1
    width_3=1：卷积结果的宽度 = width_1 - width_2 + 1 = 3-3+1

"""




torch.Size([2, 8, 6, 1])


'\n参数解释\n--input的参数：(2,1,7,3) -> (batch_size, channels, height_1, width_1)\n    batch_size=2：一个batch中的样例数\n    channels=1：通道数，也就是当前层的深度\n    height_1=7：也就是当前图片的高\n    width_1=3：也就是当前图片的宽\n    \n--Conv2d的参数：(1,8,(2,3))  -> (in_channels, out_channels, (height_2, width_2))\n    in_channels=1：输入通道数，和上面的input的channels一样\n    out_channels=8：输出的深度，也就是卷积产生的通道数\n    height_2=2：过滤器或者卷积核的高\n    width_2=2：过滤器或者卷积核的宽\n\n--res输出的参数：(2,8,6,1) -> (batch_size, out_channels, height_3, width_3)\n    batch_size=2：一个batch中的样例数\n    out_channels=8：输出通道数，也就是卷积核产生的通道数\n    height_3=6：卷积结果的高度 = height_1 - height_2 + 1 = 7-2+1\n    width_3=1：卷积结果的宽度 = width_1 - width_2 + 1 = 3-3+1\n\n'

In [25]:
import torch
import torch.nn.functional as F

#下面的两组变量是为了验证输入中的多通道in_channels在与对应的卷积核相乘后，需要都加在一起
#inputs = torch.ones(1,1,3,3)
#filters = torch.ones(1,1,2,2)

#inputs = torch.ones(1,2,3,3)
#filters = torch.ones(1,2,2,2)


#下面的两组变量是为了验证不同Batch之间计算，其实用的是同一批卷积核

#inputs = torch.ones(1,1,3,3)
#filters = torch.ones(1,1,2,2)

#inputs = torch.ones(2,1,3,3)
#filters = torch.ones(1,1,2,2)

#下面两组变量是为了验证out_channels的作用,out_channels其实对应的就是从最高维增加了卷积核；
#也就是前面参数部分解析的是输出的通道数，多了几层卷积核，也就是相当于多了几层深度
#inputs = torch.ones(1,1,3,3)
#filters = torch.ones(1,1,2,2)

inputs = torch.ones(1,1,3,3)
filters = torch.ones(2,1,2,2)

 
output = F.conv2d(inputs, filters, padding=0)
print("The output size is:", output.shape)
print("The output is:", output)

"""
首先需要保证输入的in_channels和卷积核中的in_channels是一样的，
因为多个通道的最后的那个二维矩阵与相应的卷积核相乘后，最后需要都加在一起,这也解释了为什么最后的输出中维度没有in_channels。

然后对于多个batch，他们采用的卷积核都是同一批的，只是每个batch自己的数据会发生变化。

然后对于多个out_channels，其实就是增加了几层卷积核，他们与之前的相互独立，新的计算结果相当于在最高维增加深度

"""



The output size is: torch.Size([1, 2, 2, 2])
The output is: tensor([[[[4., 4.],
          [4., 4.]],

         [[4., 4.],
          [4., 4.]]]])


'\n首先需要保证输入的in_channels和卷积核中的in_channels是一样的，\n因为多个通道的最后的那个二维矩阵与相应的卷积核相乘后，最后需要都加在一起,这也解释了为什么最后的输出中维度没有in_channels。\n\n然后对于多个batch，他们采用的卷积核都是同一批的，只是每个batch自己的数据会发生变化。\n\n'

In [28]:
import torch
import torch.nn.functional as F

inputs = torch.ones(2,2,4)
filters = torch.ones(1,2,2)
#其实它的实现以及计算过程与conv2d一样，虽然它设置的卷积核只有一个width，但是他在计算时，是按照卷积核为[in_channels, width]进行计算的
#这里的in_channel就可以看做是文本中的句子长度
#这里的out_channel可以设置多个，甚至设置的和input的最后一维一样【在文本中最后一维就是emb_dim】，这样就可以得到新的向量表示

outputs = F.conv1d(inputs, filters)

print("The output shape is:", outputs.shape)
print("The output is:", outputs)




The output shape is: torch.Size([2, 1, 3])
The output is: tensor([[[4., 4., 4.]],

        [[4., 4., 4.]]])


In [30]:
#这是conv3d的具体实现，其实这里和conv1d、conv2d是一样的
#这是他的参数，只是卷积核的维度由二维变成了三维，核心计算方法和二维是一样的


import torch
import torch.nn.functional as F


inputs = torch.ones(20,16,50,10,20)
filters = torch.ones(33,16,3,3,3)

outputs = F.conv3d(inputs, filters)
print("The outputs shape is:", outputs.shape)
print("The outputs is:", outputs)



The outputs shape is: torch.Size([20, 33, 48, 8, 18])
The outputs is: tensor([[[[[432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           ...,
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.]],

          [[432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           ...,
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.]],

          [[432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           [432., 432., 432.,  ..., 432., 432., 432.],
           ...,
           [432., 432., 432.,  ..., 432., 432., 432.]